In [18]:
# Pandas Dataframe Library
import json
import pandas as pd
import os
from IPython.display import display
import numpy as np 

In [19]:
from google.colab import drive
drive.mount('/content/drive')
os.getcwd()
os.chdir('/content/drive/MyDrive/BSP6/init_dataset_gold')   

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
train_data_gold = pd.read_json('train.json', lines=True)
val_data_gold = pd.read_json('val.json', lines=True)
test_data_gold = pd.read_json('test.json', lines=True)
print(train_data_gold.label.value_counts())
print(val_data_gold.label.value_counts())

os.chdir('/content/drive/MyDrive/BSP6/init_dataset_silver')   

train_data_silver = pd.read_json('train.json', lines=True)
val_data_silver = pd.read_json('val.json', lines=True)

1    1627
0     922
2     776
Name: label, dtype: int64
1    356
0    202
2    155
Name: label, dtype: int64


In [21]:
display(train_data_gold.describe())
display(val_data_gold.describe())
display(train_data_gold.head())

,idx,label
count,3325.000000,3325.000000
mean,1662.000000,0.956090
std,959.989149,0.713373
min,0.000000,0.000000
25%,831.000000,0.000000
50%,1662.000000,1.000000
75%,2493.000000,1.000000
max,3324.000000,2.000000


,idx,label
count,713.000000,713.000000
mean,356.000000,0.934081
std,205.969658,0.705020
min,0.000000,0.000000
25%,178.000000,0.000000
50%,356.000000,1.000000
75%,534.000000,1.000000
max,712.000000,2.000000


,idx,label,context,target
0,0,2,The UK is fucked.,>The ~~UK~~ world is fucked FTFY
1,1,0,Listen to this wisdom.,Where the Fuck did you get that up arrow?
2,2,1,"""That's different.""",You're right. The ultrasound says it's a girl....
3,3,2,"Oh fuck offffff, this is just patently untrue ...",Lol you can always spot a valid arguement when...
4,4,2,This whole sub should come to terms with reali...,I stopped reading your history after you wante...


In [22]:
from sklearn.utils import resample

def oversample_minority(df):
    # Identify the minority classes
    majority_class = df.label.value_counts().idxmax()
    minority_classes = df.label.unique().tolist()
    minority_classes.remove(majority_class)
    
    # For each minority class, resample up to the number of majority class
    df_majority = df[df.label == majority_class]
    df_minority_oversampled = []
    
    for minority_class in minority_classes:
        df_minority = df[df.label == minority_class]
        df_minority_oversampled.append(resample(df_minority, replace=True, n_samples=len(df_majority), random_state=123))

    df_oversampled = pd.concat([df_majority] + df_minority_oversampled)
    return df_oversampled

def oversample_gold_silver(gold_df, silver_df):
    # Find the count of the majority class
    majority_class_count = gold_df.label.value_counts().max()

    # Initialize the oversampled dataframe with the gold data
    gold_oversampled = gold_df.copy()

    # For each unique label in the gold data
    for label in gold_df.label.unique():
        # Calculate how many more samples we need to reach the majority_class_count
        current_count = len(gold_oversampled[gold_oversampled.label == label])
        additional_samples_needed = majority_class_count - current_count

        # If additional samples are needed
        if additional_samples_needed > 0:
            # Check the number of samples available in silver data for the class
            available_samples = len(silver_df[silver_df.label == label])
            # Adjust the number of samples to resample if not enough available
            samples_to_resample = min(available_samples, additional_samples_needed)
            additional_data = resample(silver_df[silver_df.label == label], replace=False, n_samples=samples_to_resample, random_state=123)
            gold_oversampled = pd.concat([gold_oversampled, additional_data])

    return gold_oversampled



In [23]:
# Apply oversampling
train_data_gold_oversampled = oversample_gold_silver(train_data_gold, train_data_silver)
val_data_gold_oversampled = oversample_gold_silver(val_data_gold, val_data_silver)

# Print out the counts of each class to check if oversampling was successful
print(train_data_gold_oversampled.label.value_counts())
print(val_data_gold_oversampled.label.value_counts())

# Save the oversampled data
train_data_gold_oversampled.to_json(r'/content/drive/MyDrive/BSP6/init_dataset_gold_silver_balance/train.json', orient='records', lines=True)
val_data_gold_oversampled.to_json(r'/content/drive/MyDrive/BSP6/init_dataset_gold_silver_balance/val.json', orient='records' ,lines=True)


1    1627
0    1404
2    1205
Name: label, dtype: int64
1    356
0    331
2    249
Name: label, dtype: int64


In [24]:
def undersample_majority(df):
    # Identify the majority and minority classes
    majority_class = df.label.value_counts().idxmax()
    minority_class_count = df.label.value_counts().min()
    
    # Create a new dataframe with the majority class undersampled
    df_majority_undersampled = resample(df[df.label == majority_class], replace=False, n_samples=minority_class_count, random_state=123)
    
    # Get the minority class instances
    df_minority = df[df.label != majority_class]
    
    # Combine the undersampled majority class with the minority class instances
    df_undersampled = pd.concat([df_majority_undersampled, df_minority])
    return df_undersampled


In [25]:
print(train_data_gold.label.value_counts())
print(val_data_gold.label.value_counts())

1    1627
0     922
2     776
Name: label, dtype: int64
1    356
0    202
2    155
Name: label, dtype: int64


In [26]:
train_data_gold_undersampled = undersample_majority(train_data_gold)
val_data_gold_undersampled = undersample_majority(val_data_gold)

In [27]:
train_data_gold_undersampled.to_json(r'/content/drive/MyDrive/BSP6/init_dataset_gold_balance/train.json', orient='records', lines=True)
val_data_gold_undersampled.to_json(r'/content/drive/MyDrive/BSP6/init_dataset_gold_balance/val.json', orient='records' ,lines=True)

In [29]:
print(train_data_gold_undersampled.label.value_counts())
print(val_data_gold_undersampled.label.value_counts())

0    922
1    776
2    776
Name: label, dtype: int64
0    202
1    155
2    155
Name: label, dtype: int64
